In [ ]:
from model_training import ElevatorCallsDataset, ElevatorTCNModel
from torch.utils.data import DataLoader
import torch

from sklearn.metrics import accuracy_score, precision_score, recall_score
import pandas as pd
import numpy as np

from livelossplot import PlotLosses

In [ ]:
# load model
testset = pd.read_csv('testset.csv')
testset['timestamp'] = pd.to_datetime(testset['timestamp'])

# timestamp to unix timestamp
testset['timestamp'] = testset['timestamp'].astype(np.int64) // 10**9

test_dataset = ElevatorCallsDataset(testset, input_len=60, gap=30, output_window=60)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')


    all_preds = []
    all_labels = []

    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            logits = model(x_batch)
            preds = (torch.sigmoid(logits) > 0.5).float()
            all_preds.append(preds.cpu())
            all_labels.append(y_batch.cpu())

    all_preds = torch.cat(all_preds, dim=0).numpy()
    all_labels = torch.cat(all_labels, dim=0).numpy()

    accuracy = accuracy_score(all_labels.flatten(), all_preds.flatten())
    precision = precision_score(all_labels.flatten(), all_preds.flatten())
    recall = recall_score(all_labels.flatten(), all_preds.flatten())

    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')

In [ ]:
model = ElevatorTCNModel(input_channels=len(testset.columns), output_size=len(testset.columns)-1)
model.to(device)
model.load_state_dict(torch.load('elevator_tcn_model.pth'))
model.eval()

all_preds = []
all_labels = []
accuracies = []
precisions = []
recalls = []

with torch.no_grad():
    model.eval()
    for x_batch, y_batch in test_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        logits = model(x_batch)
        preds = (torch.sigmoid(logits) > 0.5).float()
        
        all_preds.append(preds.cpu())
        all_labels.append(y_batch.cpu())
        
    for i in range(len(all_preds)):
        all_preds[i] = all_preds[i].numpy()
        all_labels[i] = all_labels[i].numpy()
        accuracies.append(accuracy_score(all_labels[i], all_preds[i]))
        precisions.append(precision_score(all_labels[i], all_preds[i], zero_division=0))
        recalls.append(recall_score(all_labels[i], all_preds[i], zero_division=0))
    
print(f'Accuracy: {np.mean(accuracies)}')
print(f'Precision: {np.mean(precisions)}')
print(f'Recall: {np.mean(recalls)}')